<a href="https://colab.research.google.com/github/algodigger/playground/blob/main/Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reranking

#### Let's check how reranking model is working in the follwoing usecase
0. Set up everything and initialize [FlagEmddings](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/reranker) model for reranking
1. Upload pdf document  (Pdf for simplicity)
2. Split to chunks and convert to openAi embeddings
3. Store embeddings in ChromaDB
4. Query the document

In [ ]:
%pip install langchain
%pip install chromadb
%pip install transformers

####Sanity check:
 Test the ranker algo on simple list of Q&As using the example from the repo

In [11]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    logits = model(**inputs, return_dict=True).logits.view(-1, )
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    probabilities = softmax(logits, dim=0).tolist()
    print(f"Scores: {logits.float()}")
    print(f"Probabilities: {probabilities}")


Scores: tensor([-5.6085,  5.7623])
Probabilities: [1.1526946764206514e-05, 0.9999884366989136]


**Observations**: probably for a simple ranking we should consider to use smaller reranker to make it more realistic for a production env